In [4]:
# IBL Brain-wide Map Dataset Download - FIXED VERSION
%load_ext autoreload
%autoreload 2

import sys
import os
from one.api import ONE

from pathlib import Path

# Test connection first
print("Setting up ONE API connection...")
try:
    from one.api import ONE
    
    # Setup ONE API
    ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
    one = ONE(password='international', silent=True)
    # Test a simple query to verify connection
    from brainwidemap import bwm_query
    df_test = bwm_query(one)
    
except Exception as e:
    print(f"Error setting up ONE API: {e}")
    print("Please check your internet connection and try again")
    raise

# Import the main functions
try:
    from dmn_bwm import get_all_PETHs, stack_concat
except Exception as e:
    print(f"Error importing dmn_bwm: {e}")
    raise

print("\n" + "="*50)
print("STARTING DATA DOWNLOAD")
print("="*50)

# Start with a smaller subset for testing (first 3 insertions)
print("Testing with first 3 insertions...")
try:
    # Get first 3 insertions for testing (reduced from 10 for faster testing)
    df_subset = df_test.head(3)
    eids_subset = df_subset[['eid', 'probe_name', 'pid']].values
    
    print(f"About to process {len(eids_subset)} probe insertions:")
    for i, (eid, probe, pid) in enumerate(eids_subset):
        print(f"  {i+1}. EID: {eid}, Probe: {probe}, PID: {pid}")
    
    get_all_PETHs(eids_plus=eids_subset, vers='concat')
    
    print("\n" + "="*50)
    print("TEST SUCCESSFUL! Ready for full download.")
    print("="*50)
    print("To download all data, uncomment the line below and run:")
    print("# get_all_PETHs()  # This will download ALL ~700 probe insertions")
    print("WARNING: Full download will take hours and require significant disk space!")
    
except Exception as e:
    print(f"Error during download: {e}")
    import traceback
    traceback.print_exc()

print("\nTo proceed with concatenation after download, run:")
print("stack_concat(ephys=False)")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setting up ONE API connection...
2025-09-01 12:45:21,131 [INFO] No newer cache available
Loading bwm_query results from fixtures/2023_12_bwm_release.csv

STARTING DATA DOWNLOAD
Testing with first 3 insertions...
About to process 3 probe insertions:
  1. EID: 6713a4a7-faed-4df2-acab-ee4e63326f8d, Probe: probe00, PID: 56f2a378-78d2-4132-b3c8-8c1ba82be598
  2. EID: 56956777-dca5-468c-87cb-78150432cc57, Probe: probe01, PID: 47be9ae4-290f-46ab-b047-952bc3a1a509
  3. EID: 56956777-dca5-468c-87cb-78150432cc57, Probe: probe00, PID: 6be21156-33b0-4f70-9a0f-65b3e3cd6d4a
Processing 3 insertions
eid 6713a4a7-faed-4df2-acab-ee4e63326f8d
1 of 3 fail 56f2a378-78d2-4132-b3c8-8c1ba82be598 - Error: trials 
 The ALF object was not found.  This may occur if the object or namespace or incorrectly formatted e.g. the object "_ibl_trials.intervals.npy" would be found with the filters `object="trials", namespace="ibl"` 
Det

Traceback (most recent call last):
  File "/Users/mitchellostrow/Desktop/Projects/playingaround/ibl_data/bwm_download/bwm_dmn/dmn_bwm.py", line 1356, in get_all_PETHs
    D = concat_PETHs(pid, vers=vers)
  File "/Users/mitchellostrow/Desktop/Projects/playingaround/ibl_data/bwm_download/bwm_dmn/dmn_bwm.py", line 399, in concat_PETHs
    trials, mask = load_trials_and_mask(one, str(eid),
  File "/Users/mitchellostrow/Desktop/Projects/playingaround/ibl_data/paper-brain-wide-map/brainwidemap/bwm_loading.py", line 288, in load_trials_and_mask
    sess_loader.load_trials()
  File "/Users/mitchellostrow/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/brainbox/io/one.py", line 1474, in load_trials
    self.trials = self.one.load_object(
  File "/Users/mitchellostrow/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py", line 166, in wrapper
    return method(self, *args, **kwargs)
  File "/Users/mitchellostrow/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util

In [3]:
stack_concat(ephys=False,get_concat=False)

combining 3 insertions for version concat
Loading bwm_query results from fixtures/2023_12_bwm_release.csv
3 insertions combined
339 good cells stacked before ephys
embedding rastermap on concat...
2025-09-01 12:43:50,258 [INFO] normalizing data across axis=1
2025-09-01 12:43:50,263 [INFO] projecting out mean along axis=0
2025-09-01 12:43:50,273 [INFO] data normalized, 0.01sec
2025-09-01 12:43:50,274 [INFO] sorting activity: 339 valid samples by 2496 timepoints
2025-09-01 12:43:50,669 [INFO] n_PCs = 200 computed, 0.41sec
2025-09-01 12:43:50,819 [INFO] 88 clusters computed, time 0.56sec
2025-09-01 12:43:56,567 [INFO] clusters sorted, time 6.31sec
2025-09-01 12:43:56,618 [INFO] clusters upsampled, time 6.36sec
2025-09-01 12:43:56,632 [INFO] rastermap complete, time 6.37sec
embedding umap on concat...
embedding pca on concat...
339 cells in eventually


In [21]:
import numpy as np
#load and make sure the data works
pth_dmn = Path(one.cache_dir, 'dmn', 'res')
datapath = Path(pth_dmn, f'concat_ephysFalse.npy')
data = np.load(datapath, allow_pickle=True).flat[0]
print(data.keys())
data['pca_z'].shape,data['umap_z'].shape

dict_keys(['ids', 'xyz', 'uuids', 'pid', 'axial_um', 'lateral_um', 'channels', 'len', 'fr', 'concat_z', 'isort', 'umap_z', 'pca_z'])


((339, 2), (339, 2))